# =====================================================================================
# <center><h1>Forecasting Spare-Part Inventory</h1></center>
# =====================================================================================

### Description

* This dataset contains vehicle service records with invoice dates, vehicle details, odometer readings, and spare part descriptions, used to analyze and predict maintenance patterns and spare parts demand.


### Import lib & models 

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

### Loading Dataset & Basic Checks

In [ ]:
# loading the dataset & checking basic info
df=pd.read_csv('Spare_part_inventory.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='O')

In [ ]:
df["invoice_date"] = pd.to_datetime(df["invoice_date"], dayfirst=True)

df = df.sort_values(["invoice_line_text", "invoice_date"])

df.reset_index(drop=True, inplace=True)

In [ ]:
daily_demand = (
    df.groupby(["invoice_line_text", "invoice_date"])
      .size()
      .reset_index(name="daily_usage")
)


In [ ]:
daily_demand["avg_usage_7d"] = (
    daily_demand.groupby("invoice_line_text")["daily_usage"]
    .transform(lambda x: x.rolling(7).mean())
)

daily_demand["avg_usage_30d"] = (
    daily_demand.groupby("invoice_line_text")["daily_usage"]
    .transform(lambda x: x.rolling(30).mean())
)

daily_demand["usage_std_30d"] = (
    daily_demand.groupby("invoice_line_text")["daily_usage"]
    .transform(lambda x: x.rolling(30).std())
)

In [ ]:
daily_demand["day_of_week"] = daily_demand["invoice_date"].dt.dayofweek
daily_demand["month"] = daily_demand["invoice_date"].dt.month
daily_demand["is_weekend"] = daily_demand["day_of_week"].isin([5,6]).astype(int)

In [ ]:
INITIAL_STOCK = 110        # assumed starting stock
SUPPLIER_LEAD_TIME = 7   # days


In [ ]:
daily_demand["simulated_stock"] = (
    INITIAL_STOCK -
    daily_demand.groupby("invoice_line_text")["daily_usage"].cumsum()
)

daily_demand["simulated_stock"] = daily_demand["simulated_stock"].clip(lower=0)


In [ ]:
daily_demand["hard_stockout"] = (daily_demand["simulated_stock"] == 0).astype(int)

In [ ]:
daily_demand["days_of_inventory"] = (
    daily_demand["simulated_stock"] /
    (daily_demand["avg_usage_7d"] + 1)
)


In [ ]:
daily_demand["lead_time_demand"] = (
    daily_demand["avg_usage_7d"] * SUPPLIER_LEAD_TIME
)

daily_demand["stockout_next_7d"] = (
    daily_demand["simulated_stock"] <= daily_demand["lead_time_demand"]
).astype(int)


In [ ]:
features = [
    "avg_usage_7d",
    "avg_usage_30d",
    "usage_std_30d",
    "days_of_inventory",
    "day_of_week",
    "month",
    "is_weekend"
]

daily_demand = daily_demand.dropna().reset_index(drop=True)


In [ ]:
daily_demand.head()

In [ ]:
daily_demand['stockout_next_7d'].value_counts()

In [ ]:
daily_demand[daily_demand['hard_stockout'] == 1]